In [1]:
import torch 
import torch.nn as nn
from typing import List, Union, Tuple, Dict, Optional

import numpy as np

from models import FusionModule

from PIL import Image
import requests
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoImageProcessor,
    AutoTokenizer,
)

from models import Fusion

d:\Apps\Anaconda3\envs\idv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vision_pretrained = "google/vit-base-patch16-224"
text_pretrained = "bert-base-uncased"

model = Fusion(text_pretrained, vision_pretrained)

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['c

In [3]:
tokenizer = AutoTokenizer.from_pretrained(text_pretrained)
image_processor = AutoImageProcessor.from_pretrained(vision_pretrained)
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)

# contrastive training
urls = [
    "http://images.cocodataset.org/val2017/000000039769.jpg",
    "https://farm3.staticflickr.com/2674/5850229113_4fe05d5265_z.jpg",
]
images = [Image.open(requests.get(url, stream=True).raw) for url in urls]
inputs = processor(
    text=["a photo of a cat", "a photo of a dog"], 
    images=images, return_tensors="pt", padding=True
)


In [4]:
outputs = model(**inputs, return_dict=True, return_loss=True)

In [7]:
outputs.logits_per_image

tensor([[7.5166, 0.7326],
        [1.2819, 8.4188]], grad_fn=<PermuteBackward0>)